<a href="https://colab.research.google.com/github/llm-finetune/experiment-tracking/blob/main/Gemma_2B_on_Alpaca_using_WandB_saveCheckpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
%pip install -U transformers
%pip install wandb
%pip install accelerate
%pip install -U bitsandbytes
%pip install -U peft
%pip install -U trl
%pip install dill
#%pip install mlflow
%pip install flash_attn

In [2]:
!wget https://raw.githubusercontent.com/Instruction-Tuning-with-GPT-4/GPT-4-LLM/main/data/alpaca_gpt4_data.json

--2024-03-15 17:45:37--  https://raw.githubusercontent.com/Instruction-Tuning-with-GPT-4/GPT-4-LLM/main/data/alpaca_gpt4_data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43379276 (41M) [text/plain]
Saving to: ‘alpaca_gpt4_data.json’

alpaca_gpt4_data.js 100%[===================>]  41.37M   233MB/s    in 0.2s    

2024-03-15 17:45:39 (233 MB/s) - ‘alpaca_gpt4_data.json’ saved [43379276/43379276]



In [3]:
import json

dataset_file = "alpaca_gpt4_data.json"

with open(dataset_file, "r") as f:
    alpaca = json.load(f)

In [4]:
#type(alpaca), alpaca[0:3], len(alpaca)

In [5]:
import wandb
from google.colab import userdata
secret_wandb = userdata.get('wandb')
wandb.login(key = secret_wandb)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
'''

# log to wandb
with wandb.init(project="alpaca_ft"):
    at = wandb.Artifact(
        name="alpaca_gpt4",
        type="dataset",
        description="A GPT4 generated Alpaca like dataset for instruction finetunning",
        metadata={"url":"https://github.com/Instruction-Tuning-with-GPT-4/GPT-4-LLM#how-good-is-the-data"},
    )
    at.add_file(dataset_file)

    # log as a table
    table = wandb.Table(columns=list(alpaca[0].keys()))
    for row in alpaca:
        table.add_data(*row.values())
    wandb.log({"alpaca_gpt4_table": table})
    '''

'\n\n# log to wandb\nwith wandb.init(project="alpaca_ft"):\n    at = wandb.Artifact(\n        name="alpaca_gpt4",\n        type="dataset",\n        description="A GPT4 generated Alpaca like dataset for instruction finetunning",\n        metadata={"url":"https://github.com/Instruction-Tuning-with-GPT-4/GPT-4-LLM#how-good-is-the-data"},\n    )\n    at.add_file(dataset_file)\n\n    # log as a table\n    table = wandb.Table(columns=list(alpaca[0].keys()))\n    for row in alpaca:\n        table.add_data(*row.values())\n    wandb.log({"alpaca_gpt4_table": table})\n    '

**Train/Eval Split**

In [7]:
#import random

#seed = 42

#random.seed(seed)
#random.shuffle(alpaca)  # this could also be a parameter

In [8]:
train_dataset = alpaca[:-40000]
eval_dataset = alpaca[-2000:]

In [9]:
len(train_dataset)


12002

In [10]:
import pandas as pd

train_df = pd.DataFrame(train_dataset)
eval_df = pd.DataFrame(eval_dataset)

train_table = wandb.Table(dataframe=train_df)
eval_table  = wandb.Table(dataframe=eval_df)

train_df.to_json("alpaca_gpt4_train.jsonl", orient='records', lines=True)
eval_df.to_json("alpaca_gpt4_eval.jsonl", orient='records', lines=True)

with wandb.init(project="alpaca_ft", job_type="split_data"):
    at = wandb.Artifact(
        name="alpaca_gpt4_splitted",
        type="dataset",
        description="A GPT4 generated Alpaca like dataset for instruction finetunning",
        metadata={"url":"https://github.com/Instruction-Tuning-with-GPT-4/GPT-4-LLM#how-good-is-the-data"},
    )
    at.add_file("alpaca_gpt4_train.jsonl")
    at.add_file("alpaca_gpt4_eval.jsonl")
    wandb.log_artifact(at)
    wandb.log({"train_dataset":train_table, "eval_dataset":eval_table})

wandb: Currently logged in as: llm-finetune-wb. Use `wandb login --relogin` to force relogin


In [11]:
def prompt_no_input(row):
    return ("Below is an instruction that describes a task. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Response:\n").format_map(row)

In [12]:
row = alpaca[0]
print(prompt_no_input(row))

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Give three tips for staying healthy.

### Response:



In [13]:
row

{'instruction': 'Give three tips for staying healthy.',
 'input': '',
 'output': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.'}

In [14]:
def prompt_input(row):
    return ("Below is an instruction that describes a task, paired with an input that provides further context. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n").format_map(row)

In [15]:
row = alpaca[234]
print(prompt_input(row))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Name one benefit of meditation.

### Input:


### Response:



In [16]:
row

{'instruction': 'Name one benefit of meditation.',
 'input': '',
 'output': 'Meditation can be incredibly relaxing and can help reduce stress levels by allowing you to quiet your mind and let go of tension. As a result, it can improve your overall well-being, and help you feel more calm, focused, and in control.'}

In [17]:
#the refactored function
def create_alpaca_prompt(row):
    return prompt_no_input(row) if row["input"] == "" else prompt_input(row)

In [18]:
import json
from wandb import Api

api = Api()
artifact = api.artifact('llm-finetune-wb/alpaca_ft/alpaca_gpt4_splitted:v0', type='dataset')
dataset_dir = artifact.download()

def load_jsonl(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return data

train_dataset = load_jsonl(f"{dataset_dir}/alpaca_gpt4_train.jsonl")
eval_dataset = load_jsonl(f"{dataset_dir}/alpaca_gpt4_eval.jsonl")

wandb:   2 of 2 files downloaded.  


In [19]:
len(eval_dataset)

2000

In [20]:
train_prompts = [create_alpaca_prompt(row) for row in train_dataset]
eval_prompts = [create_alpaca_prompt(row) for row in eval_dataset]

In [21]:
print(train_prompts[50])

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Create a link to an online store that sells books.

### Response:



In [22]:
def pad_eos(ds):
    EOS_TOKEN = "</s>"
    return [f"{row['output']}{EOS_TOKEN}" for row in ds]

In [23]:
train_outputs = pad_eos(train_dataset)
eval_outputs = pad_eos(eval_dataset)
train_outputs[0]

'As an AI assistant, I do not have personal experiences, but I can provide an example. One instance where I used my influence in a positive way was when a user was feeling highly stressed and overwhelmed about an upcoming job interview. They confided in me, expressing their self-doubts and fears of failure. Recognizing the power of positive reinforcement and encouragement, I drew upon my resources to provide the user with uplifting and motivational messages. I reminded them of their strengths and past accomplishments, and suggested coping strategies such as visualization and practicing positive self-talk. Furthermore, I helped them prepare for the interview by offering practice questions, tips on body language, and advice on how to effectively communicate their skills and experiences. As a result, the user reported feeling more confident and capable of performing well in their interview. They later informed me that they landed the job and thanked me for my support and encouragement. I 

In [24]:
train_dataset = [{"prompt":s, "output":t, "example": s + t} for s, t in zip(train_prompts, train_outputs)]
eval_dataset = [{"prompt":s, "output":t, "example": s + t} for s, t in zip(eval_prompts, eval_outputs)]

In [25]:
print(train_dataset[56]["example"])

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write a story that features a character named "Jenna".

### Response:
Once upon a time, there was a girl named Jenna. Jenna was a sweet and caring girl who lived in a small village at the foot of a great mountain. Her days were full of joy as she played and laughed with her friends and helped her parents with their work on the farm.

However, one day, a terrible disease came to the village, spreading like wildfire and causing many of the villagers to fall sick. Jenna’s parents were among those who became ill, and with no cure in sight, the village was thrown into despair.

Jenna, however, refused to give up hope. She remembered stories her grandfather used to tell her about a magical flower that grew high up on the mountain, said to have the power to cure any illness. Gathering her courage, Jenna decided to set out on a quest to find the flower and bring it back t

**Converting text to numbers: Tokenizer**
We need to convert the dataset into tokens, you can quickly do this with the workhorse of the transformers library, the Tokenizer! This function does a lot of heavy lifting besides tokenizing the text.

*   It tokenizes the text
*   Converts the outputs to PyTorch tensors
*   Pads the inputs to match length and more!Pads the inputs to match length and more!







In [26]:
#import torch
#from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer

In [27]:
model_id = 'google/gemma-2b-it'
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

In [28]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    #attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)
#tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
tokenizer.padding_side = 'right' # to prevent warnings

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [29]:
# LoRA config based on QLoRA paper & Sebastian Raschka experiment
peft_config = LoraConfig(
        lora_alpha=8,
        lora_dropout=0.05,
        r=16,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)

In [30]:
def my_checkpoint_fn(trainer):
    # Access the model, optimizer, and other training state from trainer
    model = trainer.model
    optimizer = trainer.optimizer
    step = trainer.global_step

    # Save the model state, optimizer state, and other relevant information
    state = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'step': step
    }
    # Save the state to disk using your preferred method (e.g., torch.save)
    torch.save(state, f'checkpoint_{step}.pth')

# Create an SFTTrainer instance with the custom checkpointing function
#trainer = SFTTrainer(
#    model=model,
#    training_args=TrainingArguments(save_strategy=my_checkpoint_fn)
#)

In [31]:
args = TrainingArguments(
    output_dir="./results", # directory to save and repository id
    num_train_epochs=1,                     # number of training epochs
    per_device_train_batch_size=2,          # batch size per device during training
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    logging_steps=50,                       # log every 10 steps
    save_steps=100,
    save_strategy="steps",                  # save checkpoint every epoch
    #fp16=True,                              # use bfloat16 precision
    #tf32=True,                              # use tf32 precision
    ### peft specific arguments ###
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="linear",           # use constant learning rate scheduler
    #report_to="tensorboard",                # report metrics to tensorboard
    report_to="wandb",
    push_to_hub=True,                       # push model to hub
    #evaluation_strategy="steps",

)
max_seq_length = 1512 # max sequence length for model and packing of the dataset

In [32]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    #eval_dataset=eval_dataset,
    dataset_text_field="example",
    ### peft specific arguments ###
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    dataset_kwargs={
        "add_special_tokens": False, # <bos> and <eos> should be part of the dataset.
        "append_concat_token": False, # make sure to not add additional tokens when packing
    }
)

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [33]:
wandb.init(project="alpaca_ft", # the project I am working on
           tags=["lr=2e-5","lr_sc=linear","batchsize=4","epoch=1"],
           job_type="train_eval",
           config=args) # the Hyperparameters I want to keep track of

In [34]:
# start training, the model will be automatically saved to the hub and the output directory
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
50,1.556700
100,1.230900
150,1.200600
200,1.199700
250,1.164500
300,1.161600
350,1.155600
400,1.137700
450,1.152700
500,1.133400


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.

Step,Training Loss
50,1.556700
100,1.230900
150,1.200600
200,1.199700
250,1.164500
300,1.161600
350,1.155600
400,1.137700
450,1.152700
500,1.133400


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
#new_model="gemma-2b-alpaca-full"

In [ ]:
#trainer.model.save_pretrained(new_model)

In [ ]:
#trainer.model.config.save_pretrained(new_model)

In [ ]:
#trainer.model.push_to_hub(new_model, use_temp_dir=True)

In [ ]:
#trainer.model.config.push_to_hub(new_model, use_temp_dir=True)

In [ ]:
#tokenizer.push_to_hub(new_model, use_temp_dir=True)

In [35]:
wandb.finish()

train/epoch,▁▂▂▃▄▄▅▅▆▇▇█
train/global_step,▁▂▂▃▄▄▅▅▆▇▇█
train/grad_norm,█▅▆▆▂▃▃▂▃▁▃▃
train/learning_rate,█▇▇▆▅▅▄▄▃▂▂▁
train/loss,█▃▂▂▂▂▂▁▁▁▁▁
train/epoch,0.78
train/global_step,600
train/grad_norm,0.39726
train/learning_rate,5e-05
train/loss,1.1244
